In [13]:
# import

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from datetime import date
import time
from bs4 import BeautifulSoup
import pandas as pd
import requests
import os

In [19]:
# hyperlinks

interia_hyperlink = 'https://wydarzenia.interia.pl'
onet_hyperlink = 'https://www.onet.pl/'

try:
    path = os.getcwd() + '\\' + str(date.today()) + '\\'
    os.mkdir(path)
except:
    print('Error: Directory already exists')

try:
    path = path + 'interia\\'
    os.mkdir(path)
except:
    print('Error: Directory already exists')

In [15]:
# loading interia links on page

s = Service('chromedriver.exe')
driver = webdriver.Chrome(service=s)
driver.get(interia_hyperlink)
button_div_xpath = "/html/body/div[7]/div/div/section/div"

time.sleep(1)
while True:
    try:       
        show_more_button_div = driver.find_element(By.XPATH, button_div_xpath)
        s = BeautifulSoup(show_more_button_div.get_attribute('innerHTML'))
        s2 = s.find('button')
        #print(str(s2['class']))
        
        if "hidden" in str(s2['class']):
            print("All loaded")
            break        
        
        show_more_button = driver.find_element(By.XPATH, button_div_xpath + "/button")
        driver.execute_script("arguments[0].click();", show_more_button)
        print("Show more button clicked")
        time.sleep(0.5)
    except:
        print("Error: No \"Show more\" button detected")
        break
        
html_text = driver.page_source
driver.close()


Show more button clicked
Show more button clicked
Show more button clicked
Show more button clicked
Show more button clicked
All loaded


In [20]:
# get interia links

soup = BeautifulSoup(html_text)
news_soup = soup.find_all('a', class_ = 'tile-magazine-title-url')
#print("Number of links: " + str(len(news_soup)))

d = {'Title': [], 'Address':[], 'Article text':[]}
news_DF = pd.DataFrame(data=d)

for n in news_soup:  
    address = interia_hyperlink + n['href']
    
    try:
        news_html = requests.get(address)
    
        soup = BeautifulSoup(news_html.text)
        article_container = soup.find('div', class_ = 'article-container')

        article_body = article_container.find_all('p')

        #remove block of text - useless
        article_body.pop(1)
        #remove advertisment
        article_body.pop(len(article_body)-1)
        
        #add article lines to one text
        t = ''
        for i in article_body:
        #print(i.text)
            t = t + ' ' + i.text

        #create DF
        x = pd.DataFrame(data={'Title': [n['title']], 'Address': [address], 'Article text': [t]})
        news_DF = pd.concat([news_DF, x], ignore_index = True)
        
        #save to separate files
        with open(path + str(len(news_DF.index)) + ".txt", "w", encoding="utf-8") as f:
            f.write(t)
    except:
        print("Link does not work: " + str(len(news_DF.index))) 
    
#print("Number of scraped articles: " + str(len(news_DF)))


news_DF.to_csv( path + 'News '+ str(date.today()) + '.csv', sep=';')

Link does not work: 3
Link does not work: 10
Link does not work: 50


In [23]:
import morfeusz2


x = morfeusz2.Morfeusz()
a = t
result = x.analyse(a)

print(a)
for r in result:
    print(r)

ModuleNotFoundError: No module named 'morfeusz2'